In [3]:
from requests import get, post, delete
import os
import pandas as pd
from sensitive import *


In [4]:
auth = f"Bearer {AUTH_TOKEN}"
headers = {"Accept": "application/json",
           "Content-Type": "application/json", "Authorization": auth}


NameError: name 'AUTH_TOKEN' is not defined

In [24]:
artists = {}
songs = []


def get_total_song_count():
    playlist_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}', headers=headers)
    return playlist_res.json()["tracks"]["total"]


def add_genres(list, genres):
    for genre in genres:
        if genre not in list:
            list.append(genre)

    return list


def get_song_genres(song):
    genres = []
    for artist in song["track"]["artists"]:
        id = artist["id"]
        if id not in artists:
            artist_genres_res = get(
                f'https://api.spotify.com/v1/artists/{id}', headers=headers)
            try:
                artist_genres = artist_genres_res.json()["genres"]
            except Exception:
                print(artist_genres_res.json())
            genres = add_genres(genres, artist_genres)
            artists[artist["name"]] = artist_genres
        else:
            genres = add_genres(genres, artists[id])

    return genres


def song_data(song):
    return song["track"]['id'], song["track"]['name'], song["track"]['popularity'], [artist["name"] for artist in song["track"]["artists"]]


In [25]:
total_song_count = get_total_song_count()

In [26]:


all_genres = []
for offset in range(0, total_song_count, 100):
    all_genres_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}/tracks?limit=100&offset={offset}', headers=headers)
    for song in all_genres_res.json()["items"]:
        (id, name, popularity, artist), song_genres = song_data(
            song), get_song_genres(song)
        songs.append({"id": id, "name": name, "artists": artist,
                     "popularity": popularity, "genres": song_genres})
        all_genres = add_genres(all_genres, song_genres)


In [27]:
songs = songs[-total_song_count:]

In [28]:
playlist = pd.DataFrame(data=songs, columns=[
                        'id', 'name', 'artists', 'popularity', 'genres'])


In [29]:
playlist


,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"[Justin Bieber, Dominic Fike]",57,"[canadian pop, pop, alternative pop rock]"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"[Kina, Snøw]",83,"[sad lo-fi, sad rap]"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,[Alessia Cara],77,"[canadian contemporary r&b, canadian pop, danc..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"[Teqkoi, Snøw]",69,"[lo-fi chill, sad lo-fi, sad rap]"
4,6Rf3dWdNQs5LTqFOiavqQr,Canada (feat. Alessia Cara),"[Lauv, Alessia Cara]",61,"[dance pop, electropop, pop, canadian contempo..."
...,...,...,...,...,...
991,7dMelypWXvdBG5RpRO1RJU,Purple Candy,[New Friends],54,[]
992,0sEpSjknnHdACZVc7jbmgv,i miss u,[Sammyseven],43,[]
993,01WJWHv12F3nWeXGYsmOOs,Our Way Out,[Nico Collins],57,"[alt z, social media pop]"
994,6wBsMC2U4dbl4JHl1F5z19,Summer Nights,"[Ebmusic, bobbyxyz, Kina]",40,"[sad lo-fi, sad rap]"


In [30]:
df = pd.DataFrame(data=[{'artists':artists, 'songs':songs, 'all_genres':all_genres}], columns=['artists', 'songs', 'all_genres'])

df.to_json('../util/util.json')

In [31]:
playlist.drop_duplicates(keep='first').to_csv('../util/playlist.csv', header=True, index=None)

In [6]:
df = pd.read_json('../util/util.json')

artists, songs, all_genres = df['artists'][0], df['songs'][0], df['all_genres'][0]


In [7]:
all_artists = list(artists.keys())
playlist = pd.read_csv('../util/playlist.csv')
playlist

,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"['Justin Bieber', 'Dominic Fike']",57,"['canadian pop', 'pop', 'alternative pop rock']"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"['Kina', 'Snøw']",83,"['sad lo-fi', 'sad rap']"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,['Alessia Cara'],77,"['canadian contemporary r&b', 'canadian pop', ..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"['Teqkoi', 'Snøw']",69,"['lo-fi chill', 'sad lo-fi', 'sad rap']"
4,6Rf3dWdNQs5LTqFOiavqQr,Canada (feat. Alessia Cara),"['Lauv', 'Alessia Cara']",61,"['dance pop', 'electropop', 'pop', 'canadian c..."
...,...,...,...,...,...
991,7dMelypWXvdBG5RpRO1RJU,Purple Candy,['New Friends'],54,[]
992,0sEpSjknnHdACZVc7jbmgv,i miss u,['Sammyseven'],43,[]
993,01WJWHv12F3nWeXGYsmOOs,Our Way Out,['Nico Collins'],57,"['alt z', 'social media pop']"
994,6wBsMC2U4dbl4JHl1F5z19,Summer Nights,"['Ebmusic', 'bobbyxyz', 'Kina']",40,"['sad lo-fi', 'sad rap']"


In [34]:
def genres_indexed(genres):
    indexed = []
    for all_genres_x in all_genres:

        continue_outer = False
        for genre in genres:
            index = all_genres_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


def artists_indexed(artists):
    indexed = []
    for all_artists_x in all_artists:

        continue_outer = False
        for genre in artists:
            index = all_artists_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


In [35]:
playlist["genres_indexed"] = [genres_indexed(
    eval(playlist["genres"][x])) for x in range(len(playlist["genres"]))]


In [36]:
playlist.columns = playlist.columns.map(lambda x: x.strip())

In [37]:
playlist["artists_indexed"] = [artists_indexed(
    eval(playlist["artists"][x])) for x in range(len(playlist["artists"]))]


In [38]:
playlist.columns


Index(['id', 'name', 'artists', 'popularity', 'genres', 'genres_indexed',
       'artists_indexed'],
      dtype='object')

## KNN


In [39]:
def knn_prepared_data(df):
    df = df[['id', 'name', 'genres', 'artists',
             'popularity', 'genres_indexed', 'artists_indexed']]
    list = []
    for index in range(len(df['id'])):
        list.append({'id': df['id'][index], 'name': df['name'][index], 'genres': df['genres'][index], 'artists': df['artists'][index],
                    'popularity': df['popularity'][index], 'genres_indexed': df['genres_indexed'][index], 'artists_indexed': df['artists_indexed'][index]})

    return list


In [40]:
song_dict = knn_prepared_data(playlist)

In [41]:
import operator


def getNeighbors(song, K):
    distances = []
    for song_index, song_value in enumerate(song_dict):
        if (song_index != song):
            dist = ComputeDistance(song_dict[song], song_value)
            distances.append((song_index, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append([*distances[x]])
    return neighbors


In [42]:
def list_distance(a, b):
    distance = 0
    for item_a, item_b in list(zip(a, b)):
        if item_a != item_b:
            if int(item_a) == 1:
                distance += 0.4
            else:
                distance += 0.2
        else:
            if int(item_a) == 1:
                distance -= 0.4

    return distance

In [43]:
def ComputeDistance(a, b):
    genres_distance = list_distance(a['genres_indexed'], b['genres_indexed'])
    artists_distance = list_distance(a['artists_indexed'], b['artists_indexed'])
    popularity_distance = abs(a['popularity'] - b['popularity'])
    return genres_distance + artists_distance * 0.4 + (popularity_distance * 0.005)


## Song options

### Choose one of these to get the 50 closest songs to it


In [44]:
# Set the song name below
# top_5 = get('https://api.spotify.com/v1/me/top/tracks?time_range=short_term&limit=5', headers=headers).json()
song_name = "Billie Bossa Nova"
# top_5_names = map(lambda song: song['name'],top_5['items'])
# for i in top_5_names:
#     if i in playlist['name']:
#         song_name = i
#         break

In [45]:
K = 50
item = playlist[[playlist['name'][x] ==
                 song_name for x in range(len(playlist['name']))]]
index = item.index[0]
neighbors = getNeighbors(index, K)
print(song_dict[index]['name'], song_dict[index]['artists'],
      song_dict[index]['genres'], song_dict[index]['popularity'])
for neighbor, index in zip(neighbors, range(1, K + 1)):
    print(
        f'{index}. {song_dict[neighbor[0]]["name"]} \t {song_dict[neighbor[0]]["artists"]} \t {song_dict[neighbor[0]]["genres"]} \t {song_dict[neighbor[0]]["popularity"]} \t {neighbor[1]}')


Billie Bossa Nova ['Billie Eilish'] ['art pop', 'electropop', 'pop'] 79
1. listen before i go 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 78 	 -1.3550000000000004
2. ocean eyes 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 81 	 -1.3500000000000003
3. wish you were gay 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 77 	 -1.3500000000000003
4. all the good girls go to hell 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 77 	 -1.3500000000000003
5. i love you 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 82 	 -1.3450000000000004
6. Getting Older 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 76 	 -1.3450000000000004
7. my future 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 75 	 -1.3400000000000003
8. everything i wanted 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 84 	 -1.3350000000000004
9. when the party's over 	 ['Billie Eilish'] 	 ['art pop', 'electropop', 'pop'] 	 84 	 -1.3350000000000004
10. ilom

In [46]:
def playlist_exists(name):
    request = get('https://api.spotify.com/v1/me/playlists', headers=headers).json()

    playlists = list(map(lambda playlist: (playlist['id'], playlist['name']), request['items']))

    for playlist in playlists:
        if playlist[1] == name:
            return playlist[0]
        
    return False

In [47]:
import json
playlist_name = f"{song_name!r} Related"
new_id = ""
playlist_id_found = playlist_exists(playlist_name)
if playlist_id_found:
    new_id = playlist_id_found
    
    playlist_tracks = list(map(lambda track: {'uri': track['track']['uri']}, get(f'https://api.spotify.com/v1/playlists/{new_id}/tracks', headers=headers).json()['items']))

    delete_json = delete(f'https://api.spotify.com/v1/playlists/{new_id}/tracks',  headers=headers, data=json.dumps({"tracks": playlist_tracks})).json()
    print(delete_json)
else:
    data = {"name": playlist_name, "description": f"Songs related to {song_name!r}", "public": False}
    playlist_creation = post(f'https://api.spotify.com/v1/users/{USER_ID}/playlists', headers=headers, data=json.dumps(data))
    new_id = playlist_creation.json()['id']

In [48]:
item = playlist[[playlist['name'][x] == song_name for x in range(len(playlist['name']))]]
index = item.index[0]
song_uris = f'spotify:track:{song_dict[index]["id"]}'
for neighbor in neighbors:
    song_uris += f',spotify:track:{song_dict[neighbor]["id"]}'

add_songs_req = post(f'https://api.spotify.com/v1/playlists/{new_id}/tracks?uris={song_uris}', headers=headers, data=json.dumps({}))
add_songs_req.json()

TypeError: list indices must be integers or slices, not list